In [ ]:
# Run on cmd: (venv) python app.py
# Opens at: http://127.0.0.1:7860

import os
import json
import glob
import hashlib
import time
from datetime import datetime, timezone
import gradio as gr
import notarize  

# ---------- Config / paths ----------
PROOFS_DIR = getattr(notarize, "PROOFS_DIR", "proofs")
os.makedirs(PROOFS_DIR, exist_ok=True)
ANCHORS_LOG = os.path.join(PROOFS_DIR, "anchors.log")
VCS_PATH = os.path.join(PROOFS_DIR, "vcs.json")

# ---------- Small helpers ----------
def now_iso():
    return datetime.now(timezone.utc).astimezone().isoformat()

def sha256_hex(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

def read_json(path, default=None):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return default

def write_json(path, obj):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)

def append_anchors_log(entry: dict):
    with open(ANCHORS_LOG, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

def tail_anchors(n=50):
    if not os.path.exists(ANCHORS_LOG):
        return []
    lines = [l for l in open(ANCHORS_LOG, "r", encoding="utf-8").read().splitlines() if l.strip()]
    out = []
    for l in lines[-n:]:
        try:
            out.append(json.loads(l))
        except Exception:
            continue
    return out

# ---------- Heuristic classifier ----------
def heuristic_classifier(text: str) -> dict:
    txt = (text or "").lower()
    keywords = ["shocking","unbelievable","click here","share","viral","hoax","fake","conspiracy","secret","exposed"]
    kcount = sum(1 for k in keywords if k in txt)
    exclaim_ratio = text.count("!") / max(1, len(text.split())) if text else 0
    uppercase_ratio = sum(1 for c in text if c.isupper()) / max(1, len(text)) if text else 0
    score = min(1.0, 0.08*kcount + 0.5*exclaim_ratio + 0.02*uppercase_ratio)
    verdict = "Likely Fake" if score >= 0.25 else "Likely Real"
    return {"classifier": verdict, "score": round(score, 3), "keywords_found": kcount}

def stats(text: str):
    text = text or ""
    return f"{len(text.split())} words • {len(text)} chars"

# ---------- Recent entries + chain visual ----------
def read_recent_entries(limit=8):
    out = []
    for jf in sorted(glob.glob(os.path.join(PROOFS_DIR, "*.json")), key=os.path.getmtime, reverse=True)[:limit]:
        try:
            p = read_json(jf, {})
            out.append({
                "anchor_id": p.get("anchor_id",""),
                "sha256": p.get("sha256",""),
                "timestamp": p.get("timestamp",""),
                "verdict": p.get("metadata",{}).get("classifier",""),
                "score": float(p.get("metadata",{}).get("score",0)),
            })
        except Exception:
            continue
    return out

def color_for_score(score: float) -> str:
    r = int(255 * score)
    g = int(255 * (1.0 - score))
    return f"rgb({r},{g},90)"

def chain_visual_html(last_entries):
    blocks = []
    for i, e in enumerate(last_entries):
        hue = int(140 + (e.get("score",0)*200)) % 360
        blocks.append(f"""
          <div class="block" style="--i:{i}; --glow: hsl({hue} 90% 55%); margin-right:6px;">
            <div class="b-row">
              <div class="b-tag">{e.get("verdict","?")}</div>
              <div class="b-time" style="font-size:12px;color:rgba(255,255,255,0.7)">{e.get("timestamp","")}</div>
            </div>
            <div class="b-id" style="font-size:13px">#{e.get("anchor_id","")[:10]}…</div>
            <div class="b-hash" style="font-family:monospace;font-size:12px;color:rgba(255,255,255,0.7)">{e.get("sha256","")[:20]}…</div>
          </div>
          <div class="link"><svg viewBox="0 0 120 24" preserveAspectRatio="none"><path d="M5 12 H115" /></svg></div>
        """)
    if not blocks:
        blocks = ["<div class='muted'>No anchors yet — notarize your first article to grow the chain.</div>"]
    return f"<div class='chain'>{''.join(blocks)}</div>"

# ---------- Ledger utilities for UI ----------
def build_ledger_html(filter_term: str=None, limit:int=30):
    entries = tail_anchors(limit)
    if filter_term:
        ft = filter_term.lower()
        entries = [e for e in entries if ft in json.dumps(e).lower() or ft in str(e.get("anchor_id","")).lower()]
    if not entries:
        return "<div class='muted'>No ledger entries found.</div>"
    parts = []
    for e in reversed(entries):
        title = e.get("title", e.get("anchor_id","(anchor)"))
        ts = e.get("ts", e.get("timestamp",""))
        tx = e.get("tx") or e.get("relayer_tx") or e.get("item_tx", "")
        root = e.get("merkle_root", e.get("anchor_root",""))
        typ = e.get("type","anchor")
        extra = ""
        if typ == "vc":
            extra = f"<div style='margin-top:6px;color:var(--muted)'>VC issuer: {e.get('issuer','')} → subject: {e.get('subject','')}</div>"
        elif typ == "pin":
            extra = f"<div style='margin-top:6px;color:var(--muted)'>Pinned CID: {e.get('cid','')}</div>"
        parts.append(f"""
          <div class='timeline-item'>
            <div><b>{title}</b></div>
            <div class='muted' style='font-size:13px'>root {str(root)[:18]}… • tx {str(tx)[:18]}… • {ts}</div>
            {extra}
          </div>
        """)
    return "<div class='card'>" + "".join(parts) + "</div>"

def ledger_table_rows(limit=80):
    rows = []
    entries = tail_anchors(limit)
    for e in reversed(entries):
        rows.append([
            e.get("ts", e.get("timestamp","")),
            e.get("anchor_id", e.get("anchor_id","-")),
            (e.get("title") or "")[:40],
            e.get("tx", e.get("relayer_tx", "-")),
            (e.get("merkle_root") or e.get("anchor_root",""))[:16],
        ])
    return rows

def prepare_certificate_download(aid: str):
    aid = (aid or "").strip()
    if not aid:
        return None
    jf = os.path.join(PROOFS_DIR, f"{aid}.json")
    if not os.path.exists(jf):
        return None
    p = read_json(jf, {})
    cert_path = os.path.join(PROOFS_DIR, f"{aid}.html")
    if not os.path.exists(cert_path):
        html = f"<!doctype html><html><body style='font-family:Arial,Helvetica,sans-serif'><h2>Certificate — {p.get('anchor_id')}</h2><div><strong>Timestamp:</strong> {p.get('timestamp')}</div><div><strong>SHA-256:</strong> <code>{p.get('sha256')}</code></div><hr/><pre>{json.dumps(p, indent=2)}</pre></body></html>"
        with open(cert_path, "w", encoding="utf-8") as f:
            f.write(html)
    return cert_path

# ---------- VC helpers ----------
def issue_demo_vc(issuer: str, subject: str):
    if not issuer or not subject:
        return "Provide issuer DID and subject DID."
    vcs = read_json(VCS_PATH, {})
    if vcs is None:
        vcs = {}
    vc = {
        "@context":["https://www.w3.org/2018/credentials/v1"],
        "type":["VerifiableCredential","JournalistCredential"],
        "issuer": issuer,
        "issuanceDate": now_iso(),
        "credentialSubject": {"id": subject, "role":"Verified Journalist (demo)"}
    }
    vc["proof"] = {"type":"DemoSig", "value": sha256_hex(json.dumps(vc, sort_keys=True))}
    vcs[subject] = vc
    write_json(VCS_PATH, vcs)
    append_anchors_log({"type":"vc", "issuer": issuer, "subject": subject, "sig": vc["proof"]["value"], "ts": now_iso()})
    return json.dumps(vc, indent=2)

def list_vcs_html():
    vcs = read_json(VCS_PATH, {})
    if not vcs:
        return "<div class='muted'>No VCs issued yet.</div>"
    parts = []
    for subj, vc in vcs.items():
        parts.append(f"<div style='padding:8px;border-bottom:1px solid rgba(255,255,255,0.03)'><b>{subj}</b><div class='muted' style='font-size:13px'>Issuer: {vc.get('issuer')}</div></div>")
    return "<div class='card'>" + "".join(parts) + "</div>"

# ---------- Agent (messages-based) ----------
def agent_reply_messages(history, user_msg):
    # history: list of {"role":..., "content":...}
    if history is None:
        history = []
    user_text = (user_msg or "").strip()
    if not user_text:
        history.append({"role":"assistant","content":"Please type a question about verification, certificates, or anchoring."})
        return history
    history.append({"role":"user","content": user_text})
    l = user_text.lower()
    if "verify" in l or "proof" in l or "merkle" in l:
        ans = ("To verify: recompute SHA-256 of the original content and compare with the certificate hash. "
               "Then use the Merkle proof (leaf→root recomputation) to check inclusion against the stored root in the ledger.")
    elif "ipfs" in l or "cid" in l:
        ans = ("IPFS stores snapshots by CID. In production you'd upload to IPFS and/or Arweave and include the CID in the contract metadata.")
    elif "chainlink" in l or "on-chain" in l or "tx" in l:
        ans = ("Chainlink Functions or a small smart contract can record the Merkle root on-chain (testnet). The tx hash proves a public anchor.")
    elif "vc" in l or "credential" in l:
        ans = ("Verifiable Credentials provide publisher identity. This demo issues a simple JSON-LD VC and stores a demo signature.")
    elif "how it works" in l or "explain" in l:
        ans = ("Flow: analyze text → compute leaf hash (SHA-256) → create JSON + HTML certificate → append to local ledger → optionally compute batch Merkle root and anchor on-chain.")
    else:
        ans = ("I can help with verification steps, Merkle proofs, VCs, and linking to on-chain anchoring. Ask more specifically for step-by-step.")
    history.append({"role":"assistant","content": ans})
    return history

# ---------- CSS ----------
CUSTOM_CSS = """
:root{ --bg:#0b1220; --ink:#e6edf6; --muted:#9fb0c3; --card:#0f172a; --brand:#00c2b8; --accent:#7c3aed; --line:#1f2937;}
body, .gradio-container{ background:var(--bg); color:var(--ink); font-family: ui-sans-serif, system-ui, -apple-system, "Segoe UI", Roboto, "Inter", sans-serif; }
.card{ background:var(--card); border:1px solid var(--line); border-radius:12px; padding:12px; margin-bottom:10px; }
.muted{ color:var(--muted); }
.chain{ display:flex; gap:8px; flex-wrap:wrap; }
.timeline-item{ padding:10px; border-radius:8px; background:linear-gradient(180deg, rgba(255,255,255,0.02), rgba(255,255,255,0.01)); margin-bottom:8px; }
h1.center-glow{ display:inline-block;padding:10px 22px;border-radius:12px;font-weight:900;font-size:34px;color:#fff;text-shadow: 0 0 18px rgba(124,58,237,0.9), 0 0 48px rgba(59,0,185,0.35); letter-spacing:1px; }
.badge-dot{ width:12px;height:12px;border-radius:50%; display:inline-block; margin-right:8px; vertical-align:middle;}
"""

# ---------- NAV, HERO, ABOUT content ----------
NAV_HTML = """
<div style="display:flex;align-items:center;justify-content:space-between;padding:10px 12px;border-radius:10px;margin-bottom:8px;">
  <div style="display:flex;align-items:center;gap:10px;">
    <div style="width:36px;height:36px;border-radius:8px;background:linear-gradient(90deg,#3B00B9,#6C07FF);display:flex;align-items:center;justify-content:center;font-weight:800;color:white;">V</div>
    <div style="line-height:1;">
      <div style="font-weight:800;font-size:14px;background:linear-gradient(90deg,#3B00B9,#6C07FF);-webkit-background-clip:text;background-clip:text;-webkit-text-fill-color:transparent;display:inline-block;">
        Veracity Hub <span style="font-size:11px; vertical-align:super; margin-left:6px; color:#D8CBFF; -webkit-text-fill-color:initial;">™</span>
      </div>
      <div style="font-size:12px;color:rgba(255,255,255,0.65);">Fake News Notarizer</div>
    </div>
  </div>
  <div style="color:rgba(255,255,255,0.75);font-weight:600;">AI × Decentralized</div>
</div>
"""

HERO_HTML = """
<div id="demo"></div>
<div class="card" style="margin-bottom:12px">
  <div style="display:flex; align-items:center; gap:12px;">
    <div style="font-size:20px; font-weight:800;">AI & Decentralized Proofs</div>
    <div class="muted">“We don’t just detect, we notarize.”</div>
  </div>
  <div class="muted" style="margin-top:6px">
    Paste an article, get a verdict & a tamper-evident proof (SHA-256 + timestamp) with a certificate & local ledger.
    Optional: plug the same hash into Chainlink Functions → anchor on a public testnet.
  </div>
</div>
"""

ABOUT_MD = """
## Why I built this

**Problem:** Fake news spreads quickly; simply flagging it isn't enough.  
**Solution:** Provide both explainable signals (verdict & score) and a tamper-evident notarization that third parties can verify later.

**How to use it**  
- Paste article text in *Notarize* and click Analyze & Notarize.  
- After creating multiple per-item proofs, click *Merkle Anchor Batch* to create a batch root and append anchors (simulated tx).  
- Use *IPFS Pin (sim)* to create a demo CID for a canonical snapshot.  
- Use *Issue VC (demo)* to create a simple verifiable credential for a publisher (demo only).
"""

# ---------- Ledger explanation card ----------
LEDGER_EXPLANATION_HTML = """
<div style="margin-top: 12px; font-size: 14px;">
  <div style="background: linear-gradient(135deg, rgba(0,194,255,0.15), rgba(0,194,255,0.05));
              border-left: 4px solid #38bdf8; 
              padding: 14px 16px; 
              border-radius: 10px; 
              box-shadow: 0 2px 6px rgba(0,0,0,0.08); 
              color: var(--ink);">
    <div style="font-weight: 700; color: #38bdf8; margin-bottom: 8px;">
      ℹ️ What does this mean?
    </div>
    <p style="margin: 4px 0; color: var(--ink);">
      Each notarization record in the ledger contains:
    </p>
    <ul style="margin: 6px 0 0 18px; padding: 0; color: var(--ink);">
      <li><b>anchor_id</b>: Unique ledger entry (like a transaction ID)</li>
      <li><b>sha256</b>: Cryptographic fingerprint of the article → ensures immutability</li>
      <li><b>timestamp</b>: Date & time when the record was anchored</li>
    </ul>
    <p style="margin-top: 8px; color: #38bdf8; font-weight:600;">
      ✅ Together, these make every record <b>verifiable, tamper-proof, and trustworthy</b>.
    </p>
  </div>
</div>
"""

# ---------- Build Gradio app ----------
with gr.Blocks(css=CUSTOM_CSS, title="Fake News Notarizer — Pro") as demo:
    gr.HTML(NAV_HTML)
    gr.HTML('<div style="text-align:center;margin:14px 0;"><h1 class="center-glow">FAKE NEWS NOTARIZER</h1></div>')
    gr.HTML(HERO_HTML)

    with gr.Tabs():
        # ---------------- Notarize Tab ----------------
        with gr.TabItem("🔍 Notarize"):
            with gr.Row():
                # Left column: paste, live stats, ledger explanation, chain, raw tail
                with gr.Column(scale=7):
                    txt = gr.Textbox(
                        label="Paste article text",
                        placeholder="Paste the article text you want to verify & notarize...",
                        lines=14, autofocus=True
                    )
                    live_stats = gr.Markdown("0 words • 0 chars")
                    analyze_btn = gr.Button("Analyze & Notarize", variant="primary")
                    verdict_html = gr.HTML()

                    # Ledger explanation card 
                    gr.HTML(LEDGER_EXPLANATION_HTML)

                    # Chain visualization and raw ledger tail
                    chain_html = gr.HTML(chain_visual_html(read_recent_entries()))
                    anchors_tail = gr.Textbox(label="Ledger tail (anchors.log)", lines=10)

                # Right column: details, anchor id, actions
                with gr.Column(scale=5):
                    details_html = gr.HTML()
                    anchor_id = gr.Textbox(label="Anchor ID", interactive=False)
                    ts = gr.Textbox(label="Timestamp", interactive=False)
                    json_file = gr.File(label="Proof (JSON)")
                    cert_file = gr.File(label="Certificate (HTML)")

                    gr.Markdown("### Extra demo actions")
                    batch_anchor_btn = gr.Button("Merkle Anchor Batch (all local proofs)")
                    batch_anchor_status = gr.Markdown("")
                    ipfs_pin_btn = gr.Button("IPFS Pin (sim) for Anchor ID above")
                    ipfs_status = gr.Markdown("")
                    issue_vc_btn = gr.Button("Issue VC (demo)")
                    issue_vc_json = gr.Code(label="VC JSON", language="json")
                    relayer_btn = gr.Button("Simulate Gasless Relayer Anchor")
                    relayer_status = gr.Markdown("")

        # ---------------- Ledger & Insights Tab ----------------
        with gr.TabItem("📜 Ledger & Insights"):
            with gr.Row():
                with gr.Column(scale=7):
                    gr.Markdown("### Ledger: Chain-of-evidence")
                    ledger_search = gr.Textbox(label="Filter ledger (anchor id, tx, title...)")
                    ledger_refresh = gr.Button("Refresh Ledger")
                    ledger_html = gr.HTML(build_ledger_html(None, limit=30))
                with gr.Column(scale=5):
                    gr.Markdown("### Ledger Table")
                    ledger_table = gr.Dataframe(headers=["Timestamp","Anchor ID","Title","Tx","Root"], row_count=10, interactive=False)
                    download_anchor_id = gr.Textbox(label="Anchor ID to generate certificate")
                    download_cert_btn = gr.Button("Generate Certificate (HTML)")
                    cert_download_file = gr.File(label="Certificate file")

        # ---------------- Merkle & IPFS Tab ----------------
        with gr.TabItem("🧾 Merkle & IPFS"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Merkle batching & IPFS (demo)")
                    gr.Markdown("Use *Merkle Anchor Batch* to compute a Merkle root over all local proofs. IPFS Pin (sim) generates a demo CID stored in the ledger.")
                with gr.Column():
                    gr.Markdown("### Current VCs")
                    vcs_html = gr.HTML(list_vcs_html())

        # ---------------- VC Tab ----------------
        with gr.TabItem("🎫 VC (Demo)"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Issue a Verifiable Credential (demo)")
                    vc_issuer = gr.Textbox(label="Issuer DID (demo)", value="did:demo:issuer")
                    vc_subject = gr.Textbox(label="Subject DID (reporter)", value="did:demo:reporter")
                    issue_vc_manual_btn = gr.Button("Issue VC (with these DIDs)")
                    issue_vc_manual_out = gr.Code(language="json")

        # ---------------- AI Agent Tab ----------------
        with gr.TabItem("🤖 AI Agent"):
            gr.Markdown("### Notarizer Agent: Ask about verification, Merkle proofs, VCs, or on-chain anchoring.")
            agent_state = gr.State([])  # list of {"role","content"}
            chat = gr.Chatbot(label="Notarizer Agent", type="messages")
            agent_in = gr.Textbox(placeholder="Ask about verification, certificates, or on-chain anchoring…")
            agent_send = gr.Button("Ask")
            with gr.Row():
                faq_verify = gr.Button("How to verify")
                faq_merkle = gr.Button("Explain Merkle proof")
                faq_chainlink = gr.Button("How to anchor on-chain (summary)")

        # ---------------- About Tab ----------------
        with gr.TabItem("ℹ️ About"):
            gr.Markdown(ABOUT_MD)
            gr.Markdown("### Features quick guide")
            gr.Markdown(""" - **📰 Notarize Article:** Paste any news article or text, and the system will analyze it, generate a unique cryptographic hash, and anchor it for authenticity.  
- **🌐 Merkle Anchor Batch:** Groups multiple notarized items together into a Merkle tree, producing a single root hash — this proves integrity across many records at once.  
- **📦 IPFS Pin (Simulated):** Demonstrates how notarized articles could be pinned to decentralized storage (IPFS). You’ll see a demo CID (content ID) generated as proof of availability.  
- **🎫 Issue VC (Demo):** Issues a simulated Verifiable Credential (VC) for publishers. This shows how media organizations can get a blockchain-style certificate proving their publishing rights.  
""")

    # ---------- Bindings ----------
    txt.change(fn=stats, inputs=txt, outputs=live_stats)

    def analyze_and_notarize_front(text):
        text = (text or "").strip()
        if not text:
            return gr.update(value="Paste some text ↑"), gr.update(value=""), "", "", None, None, gr.update(value=""), gr.update(value=chain_visual_html(read_recent_entries()))
        meta = heuristic_classifier(text)
        # Use your notarize module to create the proof (expects create_proof to return anchor_id)
        anchor_id_val = notarize.create_proof(text, meta)
        proof = notarize.load_proof(anchor_id_val) or {}
        sha = proof.get("sha256","")
        ts_val = proof.get("timestamp","")
        score = float(meta["score"])
        verdict = meta["classifier"]

        badge_html = f"""
        <div style="padding:10px;border-radius:10px;background:linear-gradient(90deg, rgba(124,58,237,0.14), rgba(0,194,184,0.08));">
          <div style="display:flex;gap:12px;align-items:center">
            <div class="badge-dot" style="background:{color_for_score(score)};box-shadow:0 0 12px {color_for_score(score)}"></div>
            <div><b>{verdict}</b><div style="font-size:13px;color:var(--muted)">score {score:.3f} • keywords {meta['keywords_found']}</div></div>
          </div>
        </div>
        """

        cert_path = os.path.join(PROOFS_DIR, f"{anchor_id_val}.html")
        json_path = os.path.join(PROOFS_DIR, f"{anchor_id_val}.json")
        details_html_val = f"""
          <div class="card">
            <div><b>Anchor ID</b>: {anchor_id_val}</div>
            <div class="muted">Timestamp: {ts_val}</div>
            <div class="muted">SHA-256: {sha[:18]}…</div>
          </div>
        """
        tail = ""
        if os.path.exists(ANCHORS_LOG):
            try:
                tail = "\n".join(open(ANCHORS_LOG, "r", encoding="utf-8").read().splitlines()[-14:])
            except Exception:
                tail = ""
        chain_html_val = chain_visual_html(read_recent_entries())
        return gr.update(value=badge_html), gr.update(value=details_html_val), anchor_id_val, ts_val, (json_path if os.path.exists(json_path) else None), (cert_path if os.path.exists(cert_path) else None), gr.update(value=tail), gr.update(value=chain_html_val)

    analyze_btn.click(
        analyze_and_notarize_front,
        inputs=[txt],
        outputs=[verdict_html, details_html, anchor_id, ts, json_file, cert_file, anchors_tail, chain_html],
        show_progress=True,
    )

    # Batch anchor all (Merkle root over all local proofs)
    def batch_anchor_all_ui():
        json_files = sorted(glob.glob(os.path.join(PROOFS_DIR, "*.json")), key=os.path.getmtime)
        leaves = []
        items = []
        for jf in json_files:
            p = read_json(jf, {})
            if p and p.get("sha256"):
                leaves.append(p["sha256"])
                items.append(p)
        if not leaves:
            return "No local proofs found (run Analyze & Notarize)."
        cur = leaves[:]
        while len(cur) > 1:
            if len(cur) % 2 == 1:
                cur.append(cur[-1])
            nxt = []
            for i in range(0, len(cur), 2):
                a = bytes.fromhex(cur[i])
                b = bytes.fromhex(cur[i+1])
                nxt.append(hashlib.sha256(a + b).hexdigest())
            cur = nxt
        root = cur[0]
        tx = "0x" + sha256_hex(root + str(time.time()))[:64]
        ts_val = now_iso()
        for it in items:
            rec = {"anchor_id": it.get("anchor_id"), "title": it.get("short_text","")[:120], "sha256": it.get("sha256"), "merkle_root": root, "tx": tx, "ts": ts_val}
            append_anchors_log(rec)
        return f"Anchored {len(items)} items — tx {tx[:12]}…"

    batch_anchor_btn.click(batch_anchor_all_ui, inputs=None, outputs=[batch_anchor_status])

    # IPFS pin (sim)
    def ipfs_pin_ui(anchor_id_val):
        aid = (anchor_id_val or "").strip()
        if not aid:
            return "Enter Anchor ID after notarizing."
        jf = os.path.join(PROOFS_DIR, f"{aid}.json")
        if not os.path.exists(jf):
            return "Anchor JSON not found locally."
        data = read_json(jf, {})
        content = data.get("short_text","") + data.get("sha256","")
        cid = "bafy" + sha256_hex("cid:" + content)[:48]
        pin_record = {"anchor_id": aid, "cid": cid, "ts": now_iso()}
        write_json(os.path.join(PROOFS_DIR, f"{aid}.cid.json"), pin_record)
        append_anchors_log({"type":"pin", "anchor_id": aid, "cid": cid, "ts": now_iso()})
        return f"Pinned (sim): {cid}"

    ipfs_pin_btn.click(ipfs_pin_ui, inputs=[anchor_id], outputs=[ipfs_status])

    # Issue VC (demo) — quick button
    def issue_vc_ui():
        issuer = "did:demo:issuer"
        subject = "did:demo:reporter"
        vcs = read_json(VCS_PATH, {})
        if vcs is None:
            vcs = {}
        vc = {"@context":["https://www.w3.org/2018/credentials/v1"], "type":["VerifiableCredential","JournalistCredential"], "issuer": issuer, "issuanceDate": now_iso(), "credentialSubject": {"id": subject, "role":"Verified Reporter (demo)"}}
        vc["proof"] = {"type":"DemoSig", "value": sha256_hex(json.dumps(vc, sort_keys=True))}
        vcs[subject] = vc
        write_json(VCS_PATH, vcs)
        append_anchors_log({"type":"vc","issuer": issuer, "subject": subject, "sig": vc["proof"]["value"], "ts": now_iso()})
        return json.dumps(vc, indent=2)

    issue_vc_btn.click(issue_vc_ui, inputs=None, outputs=[issue_vc_json])

    # Manual VC issuance (from VC tab)
    def issue_vc_manual(issuer, subject):
        if not issuer or not subject:
            return "Provide both issuer and subject DID."
        return issue_demo_vc(issuer, subject)

    issue_vc_manual_btn.click(issue_vc_manual, inputs=[vc_issuer, vc_subject], outputs=[issue_vc_manual_out])

    # Relayer (simulate gasless)
    def relayer_sim_ui():
        entries = tail_anchors(200)
        latest_root = ""
        for e in reversed(entries):
            if e.get("merkle_root"):
                latest_root = e.get("merkle_root")
                break
            if e.get("anchor_root"):
                latest_root = e.get("anchor_root")
                break
        if not latest_root:
            return "No merkle root found in ledger (run batch anchor first)."
        rel_tx = "0x" + sha256_hex("relayer:" + latest_root + str(time.time()))[:64]
        append_anchors_log({"type":"relayer","anchor_root": latest_root, "relayer_tx": rel_tx, "ts": now_iso()})
        return f"Relayer submitted tx {rel_tx[:14]}…"

    relayer_btn.click(relayer_sim_ui, inputs=None, outputs=[relayer_status])

    # Ledger bindings
    ledger_refresh.click(lambda: build_ledger_html(None, limit=80), inputs=None, outputs=[ledger_html])
    ledger_refresh.click(lambda: ledger_table_rows(80), inputs=None, outputs=[ledger_table])
    ledger_search.change(lambda q: build_ledger_html(q, limit=80), inputs=[ledger_search], outputs=[ledger_html])

    download_cert_btn.click(prepare_certificate_download, inputs=[download_anchor_id], outputs=[cert_download_file])

    # Agent bindings using messages format
    agent_send.click(agent_reply_messages, inputs=[agent_state, agent_in], outputs=[agent_state]) \
        .then(lambda h: h, inputs=agent_state, outputs=chat) \
        .then(lambda: "", None, agent_in)

    # FAQ quick buttons
    def faq_prompt(history, q):
        return agent_reply_messages(history or [], q)
    faq_verify.click(lambda history: faq_prompt(history, "How do I verify a certificate step-by-step?"), inputs=[agent_state], outputs=[agent_state]) \
        .then(lambda h: h, inputs=agent_state, outputs=chat)
    faq_merkle.click(lambda history: faq_prompt(history, "Explain how Merkle proofs work for batches."), inputs=[agent_state], outputs=[agent_state]) \
        .then(lambda h: h, inputs=agent_state, outputs=chat)
    faq_chainlink.click(lambda history: faq_prompt(history, "How would I anchor a root on-chain (summary)?"), inputs=[agent_state], outputs=[agent_state]) \
        .then(lambda h: h, inputs=agent_state, outputs=chat)

    # Demo initial loads
    demo.load(lambda: chain_visual_html(read_recent_entries()), None, chain_html)
    demo.load(lambda: "\n".join(open(ANCHORS_LOG,"r",encoding="utf-8").read().splitlines()[-20:]) if os.path.exists(ANCHORS_LOG) else "", None, anchors_tail)
    demo.load(lambda: build_ledger_html(None, limit=30), None, ledger_html)
    demo.load(lambda: ledger_table_rows(30), None, ledger_table)
    demo.load(lambda: list_vcs_html(), None, vcs_html)

if __name__ == "__main__":
    demo.queue().launch(server_name="127.0.0.1", server_port=7860, show_error=True, share=True)
